In [1]:
import os
import sys
import warnings
import time
from pynq import PL
from pynq import Overlay


BF = os.path.join("./data/system_wrapper.bit")

# Download the bitfile

if not os.path.exists(BF):
    print ("%s Doesn't exist!" % BF)
else:
    print ("Found bit file!")
    
    ol = Overlay(BF)
    ol.download()

    print ("Bitfile downloaded!")



Found bit file!
Bitfile downloaded!


In [7]:
import os,sys,inspect
currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
sys.path.insert(0,parentdir) 

import warnings
import time
from pynq import PL
from pynq import Overlay
from pynq import allocate
from pynq.lib.dma import DMA

from drivers.demo_axi_streams_driver import DemoAXIStreamsDriver


das = DemoAXIStreamsDriver('demo_axi_streams_0', debug = False)
print ("Demo AXIStreamsDriver Version: 0x%08X" % das.get_version())

# Setup the DMA Writer and Reader
BUFFER_SIZE = 16
dma_writer = DMA(PL.ip_dict["axi_dma_writer"])
dma_reader = DMA(PL.ip_dict["axi_dma_reader"])
sbuffer = allocate([1, BUFFER_SIZE], dtype='u4')
rbuffer = allocate([1, BUFFER_SIZE], dtype='u4')

for i in range(BUFFER_SIZE):
    sbuffer[0][i] = i
    rbuffer[0][i] = 0

print ("sbuffer before transaction: %s" % str(sbuffer))
print ("rbuffer before transaction: %s" % str(rbuffer))
print ("")

dma_writer.set_up_tx_channel()
dma_writer.sendchannel.transfer(sbuffer, start = 0, nbytes=BUFFER_SIZE * 4)
dma_writer.sendchannel.start()
dma_writer.sendchannel.wait()


dma_reader.set_up_rx_channel()
buf = [0] * BUFFER_SIZE
dma_reader.recvchannel.transfer(rbuffer, start = 0, nbytes=BUFFER_SIZE * 4)
dma_reader.recvchannel.start()
dma_reader.recvchannel.wait()

print ("rbuffer after transaction: %s" % str(rbuffer))

if sbuffer is not None:
    sbuffer.freebuffer()

if rbuffer is not None:
    rbuffer.freebuffer()

Demo AXIStreamsDriver Version: 0x10000000
sbuffer before transaction: [[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15]]
rbuffer before transaction: [[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]]

rbuffer after transaction: [[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15]]
